In [2]:
import codecs
import csv
import itertools
import json
import re
import sys
import time
import warnings
from datetime import datetime
from urllib import parse as urlparse

from requests import RequestException
from requests_html import HTML, HTMLSession

__all__ = ['get_posts', 'write_posts_to_csv']


_base_url = 'https://m.facebook.com'
_user_agent = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
               "AppleWebKit/537.36 (KHTML, like Gecko) "
               "Chrome/76.0.3809.87 Safari/537.36")
_cookie = ('locale=en_US;')
_headers = {'User-Agent': _user_agent, 'Accept-Language': 'en-US,en;q=0.5', 'cookie': _cookie}

_session = None
_timeout = None

_likes_regex = re.compile(r'like_def[^>]*>([0-9,.]+)')
_comments_regex = re.compile(r'cmt_def[^>]*>([0-9,.]+)')
_shares_regex = re.compile(r'([0-9,.]+)\s+Shares', re.IGNORECASE)
_link_regex = re.compile(r"href=\"https:\/\/lm\.facebook\.com\/l\.php\?u=(.+?)\&amp;h=")

_cursor_regex = re.compile(r'href:"(/page_content[^"]+)"')  # First request
_cursor_regex_2 = re.compile(r'href":"(\\/page_content[^"]+)"')  # Other requests
_cursor_regex_3 = re.compile(r'\shref="(\/groups\/[^"]+bac=[^"]+)"')  # for Group requests

_photo_link = re.compile(r"href=\"(/[^\"]+/photos/[^\"]+?)\"")
_image_regex = re.compile(r"<a href=\"([^\"]+?)\" target=\"_blank\" class=\"sec\">View Full Size<\/a>", re.IGNORECASE)
_image_regex_lq = re.compile(r"background-image: url\('(.+)'\)")
_post_url_regex = re.compile(r'/story.php\?story_fbid=')

_more_url_regex = re.compile(r'(?<=…\s)<a href="([^"]+)')
_post_story_regex = re.compile(r'href="(\/story[^"]+)" aria')

_shares_and_reactions_regex = re.compile(
    r'<script>.*bigPipe.onPageletArrive\((?P<data>\{.*RelayPrefetchedStreamCache.*\})\);.*</script>')
_bad_json_key_regex = re.compile(r'(?P<prefix>[{,])(?P<key>\w+):')


def get_posts(account=None, group=None, **kwargs):
    valid_args = sum(arg is not None for arg in (account, group))

    if valid_args != 1:
        raise ValueError("You need to specify either account or group")

    if account is not None:
        path = f'{account}/posts/'
        return _get_page_posts(path, **kwargs)

    elif group is not None:
        path = f'groups/{group}/'
        return _get_group_posts(path, **kwargs)


def _get_page_posts(path, pages=10, timeout=5, sleep=0, credentials=None, extra_info=False):
    """Gets posts for a given account."""
    global _session, _timeout

    url = f'{_base_url}/{path}'

    _session = HTMLSession()
    _session.headers.update(_headers)

    if credentials:
        print(*credentials)
        _login_user(*credentials)

    _timeout = timeout

    response = _session.get(url, timeout=_timeout)
    html = HTML(html=response.html.html.replace('<!--', '').replace('-->', ''))
    cursor_blob = html.html

    while True:
        for article in html.find('article'):
            post = _extract_post(article)
            if extra_info:
                post = fetch_share_and_reactions(post)
            yield post

        pages -= 1
        if pages <= 0:
            return

        cursor = _find_cursor(cursor_blob)
        next_url = f'{_base_url}{cursor}'

        if sleep:
            time.sleep(sleep)

        try:
            response = _session.get(next_url, timeout=timeout)
            response.raise_for_status()
            data = json.loads(response.text.replace('for (;;);', '', 1))
        except (RequestException, ValueError):
            return

        for action in data['payload']['actions']:
            if action['cmd'] == 'replace':
                html = HTML(html=action['html'], url=_base_url)
            elif action['cmd'] == 'script':
                cursor_blob = action['code']


def _get_group_posts(path, pages=10, timeout=5, sleep=0, credentials=None, extra_info=False):
    """Gets posts for a given account."""
    global _session, _timeout

    url = f'{_base_url}/{path}'

    _session = HTMLSession()
    _session.headers.update(_headers)

    if credentials:
        _login_user(*credentials)

    _timeout = timeout

    while True:
        response = _session.get(url, timeout=_timeout)
        response.raise_for_status()
        html = HTML(html=response.html.html.replace('<!--', '').replace('-->', ''))
        cursor_blob = html.html

        for article in html.find('article'):
            post = _extract_post(article)
            if extra_info:
                post = fetch_share_and_reactions(post)
            yield post

        pages -= 1
        if pages <= 0:
            return

        cursor = _find_cursor(cursor_blob)

        if cursor is not None:
            url = f'{_base_url}{cursor}'

        if sleep:
            time.sleep(sleep)


def _extract_post(article):
    text, post_text, shared_text, fail = _extract_text(article)
    return {
        'fail': fail,
        'post_id': _extract_post_id(article),
        'text': text,
        'post_text': post_text,
        'shared_text': shared_text,
        'time': _extract_time(article),
        'image': _extract_image(article),
        'likes': _find_and_search(article, 'footer', _likes_regex, _parse_int) or 0,
        'comments': _find_and_search(article, 'footer', _comments_regex, _parse_int) or 0,
        'shares':  _find_and_search(article, 'footer', _shares_regex, _parse_int) or 0,
        'post_url': _extract_post_url(article),
        'link': _extract_link(article),
    }


def _extract_post_id(article):
    try:
        data_ft = json.loads(article.attrs['data-ft'])
        return data_ft['mf_story_key']
    except (KeyError, ValueError):
        return None


def _extract_text(article):
    # Open this article individually because not all content is fully loaded when skimming through pages
    # This ensures the full content can be read
    fail = False
    hasMore = _more_url_regex.search(article.html)
    if hasMore:
        match = _post_story_regex.search(article.html)
        if match:
            url = f'{_base_url}{match.groups()[0].replace("&amp;", "&")}'
            response = _session.get(url, timeout=_timeout)
            article = response.html.find('.story_body_container', first=True)
    if (hasattr(article, 'find')):
        nodes = article.find('p, header')
        if nodes:
            post_text = []
            shared_text = []
            ended = False
            for node in nodes[1:]:
                if node.tag == "header":
                    ended = True

                # Remove '... More'
                # This button is meant to display the hidden text that is already loaded
                # Not to be confused with the 'More' that opens the article in a new page
                if node.tag == "p":
                    node = HTML(html=node.html.replace('>… <', '><', 1).replace('>More<', '', 1))

                if not ended:
                    post_text.append(node.text)
                else:
                    shared_text.append(node.text)

            text = '\n'.join(itertools.chain(post_text, shared_text))
            post_text = '\n'.join(post_text)
            shared_text = '\n'.join(shared_text)

            return text, post_text, shared_text, fail
    fail = True
    text = ''
    post_text = ''
    shared_text = ''
    return text, post_text, shared_text, fail


def _extract_time(article):
    try:
        data_ft = json.loads(article.attrs['data-ft'])
        page_insights = data_ft['page_insights']
    except (KeyError, ValueError):
        return None

    for page in page_insights.values():
        try:
            timestamp = page['post_context']['publish_time']
            return timestamp
        except (KeyError, ValueError):
            continue
    return None


def _extract_photo_link(article):
    match = _photo_link.search(article.html)
    if not match:
        return None

    url = f"{_base_url}{match.groups()[0]}"

    response = _session.get(url, timeout=_timeout)
    html = response.html.html
    match = _image_regex.search(html)
    if match:
        return match.groups()[0].replace("&amp;", "&")
    return None


def _extract_image(article):
    image_link = _extract_photo_link(article)
    if image_link is not None:
        return image_link
    return _extract_image_lq(article)


def _extract_image_lq(article):
    story_container = article.find('div.story_body_container', first=True)
    if story_container is None:
        return None
    other_containers = story_container.xpath('div/div')

    for container in other_containers:
        image_container = container.find('.img', first=True)
        if image_container is None:
            continue

        style = image_container.attrs.get('style', '')
        match = _image_regex_lq.search(style)
        if match:
            return _decode_css_url(match.groups()[0])

    return None


def _extract_link(article):
    html = article.html
    match = _link_regex.search(html)
    if match:
        return urlparse.unquote(match.groups()[0])
    return None


def _extract_post_url(article):
    query_params = ('story_fbid', 'id')

    elements = article.find('header a')
    for element in elements:
        href = element.attrs.get('href', '')
        match = _post_url_regex.match(href)
        if match:
            path = _filter_query_params(href, whitelist=query_params)
            return f'{_base_url}{path}'

    return None


def _find_and_search(article, selector, pattern, cast=str):
    container = article.find(selector, first=True)
    match = container and pattern.search(container.html)
    return match and cast(match.groups()[0])


def _find_cursor(text):
    match = _cursor_regex.search(text)
    if match:
        return match.groups()[0]

    match = _cursor_regex_2.search(text)
    if match:
        value = match.groups()[0]
        return value.encode('utf-8').decode('unicode_escape').replace('\\/', '/')

    match = _cursor_regex_3.search(text)
    if match:
        value = match.groups()[0]
        return value.encode('utf-8').decode('unicode_escape').replace('\\/', '/')

    return None


def _parse_int(value):
    return int(''.join(filter(lambda c: c.isdigit(), value)))


def _decode_css_url(url):
    url = re.sub(r'\\(..) ', r'\\x\g<1>', url)
    url, _ = codecs.unicode_escape_decode(url)
    return url


def _filter_query_params(url, whitelist=None, blacklist=None):
    def is_valid_param(param):
        if whitelist is not None:
            return param in whitelist
        if blacklist is not None:
            return param not in blacklist
        return True  # Do nothing

    parsed_url = urlparse.urlparse(url)
    query_params = urlparse.parse_qsl(parsed_url.query)
    query_string = urlparse.urlencode(
        [(k, v) for k, v in query_params if is_valid_param(k)]
    )
    return urlparse.urlunparse(parsed_url._replace(query=query_string))


def _login_user(email, password):
    login_page = _session.get(_base_url)
    login_action = login_page.html.find('#login_form', first=True).attrs.get('action')
    _session.post(_base_url + login_action, data={'email': email, 'pass': password})
    if 'c_user' not in _session.cookies:
        warnings.warn('login unsuccessful')


def fetch_share_and_reactions(post: dict):
    """Fetch share and reactions information with a existing post obtained by `get_posts`.
    Return a merged post that has some new fields including `reactions`, `w3_fb_url`, `fetched_time`,
        and reactions fields `LIKE`, `ANGER`, `SORRY`, `WOW`, `LOVE`, `HAHA` if exist.

    Note that this method will raise one http request per post, use it when you want some more information.

    Example:
    ```
    for post in get_posts('fanpage'):
        more_info_post = fetch_share_and_reactions(post)
        print(more_info_post)
    ```
    """
    url = post.get('post_url')
    if url:
        w3_fb_url = urlparse.urlparse(url)._replace(netloc='www.facebook.com').geturl()
        resp = _session.get(w3_fb_url, timeout=_timeout)
        for item in _parse_share_and_reactions(resp.text):
            data = (item['jsmods']['pre_display_requires'][0][3][1]['__bbox']['result']
                    ['data']['feedback'])
            if data['subscription_target_id'] == post['post_id']:
                return {
                    **post,
                    'shares': data['share_count']['count'],
                    'likes': data['reactors']['count'],
                    'reactions': {
                        reaction['node']['reaction_type'].lower(): reaction['reaction_count']
                        for reaction in data['top_reactions']['edges']
                    },
                    'comments': data['comment_count']['total_count'],
                    'w3_fb_url': data['url'],
                    'fetched_time': int(datetime.timestamp(datetime.now())),
                }
    return post


def _parse_share_and_reactions(html: str):
    bad_jsons = _shares_and_reactions_regex.findall(html)
    for bad_json in bad_jsons:
        good_json = _bad_json_key_regex.sub(r'\g<prefix>"\g<key>":', bad_json)
        yield json.loads(good_json)


def write_posts_to_csv(account=None, group=None, filename=None, **kwargs):
    """
    :param account:     Facebook account name e.g. "nike", string
    :param group:       Facebook group id
    :param filename:    File name, defaults to <<account_posts.csv>>
    :param pages:       Number of pages to scan, integer
    :param timeout:     Session response timeout in seconds, integer
    :param sleep:       Sleep time in s before every call, integer
    :param credentials: Credentials for login - username and password, tuple
    :return:            CSV written in the same location with <<account_name>>_posts.csv
    """
    list_of_posts = list(get_posts(account=account, group=group, **kwargs))

    if not list_of_posts:
        print("Couldn't get any posts.", file=sys.stderr)
        return

    keys = list_of_posts[0].keys()

    if filename is None:
        filename = str(account or group) + "_posts.csv"

    with open(filename, 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(list_of_posts)


def _main():
    """facebook-scraper entry point when used as a script"""
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument('account', type=str, help="Facebook account")
    parser.add_argument('-f', '--filename', type=str, help="Output filename")
    parser.add_argument('-p', '--pages', type=int, help="Number of pages to download", default=10)

    args = parser.parse_args()

    write_posts_to_csv(account=args.account, filename=args.filename, pages=args.pages)


# if __name__ == '__main__':
#     _main()


In [3]:

import json

def extract_page_public_posts(page_name, count):
    max_total_page = int(count / 2)
    file_path = page_name + '_' + str(count) + '.json'
    print('Saving to ' + file_path + '...')
    data = []
    _count = 0
    try:
        for post in get_posts(page_name, pages=max_total_page, extra_info=True, timeout=15):
            print(post)
            if not (post['fail']):
                del post['fail']
                data.append(post)
                _count = _count+1
                print(_count)
                print('\n')
            if (_count >= count):
                break
    except Exception as e:
        print(e)
    finally:
        with open(file_path, "w", encoding='utf-8') as jsonfile:
            json.dump(data, jsonfile, ensure_ascii=False)


extract_page_public_posts('PTITHCM.CONFESSIONS', 100000)


Saving to PTITHCM.CONFESSIONS_100000.json...
{'fail': False, 'post_id': '3584167855043403', 'text': 'Post này là nơi để các bạn thỏa sức nêu ra ý kiến cá nhân, những đóng góp để page PTITHCM Confessions ngày càng phát triển hơn nữa.\nChân thành cảm ơn mọi người đã theo dõi tụi mình trong suốt thời gian qua nhé!', 'post_text': 'Post này là nơi để các bạn thỏa sức nêu ra ý kiến cá nhân, những đóng góp để page PTITHCM Confessions ngày càng phát triển hơn nữa.\nChân thành cảm ơn mọi người đã theo dõi tụi mình trong suốt thời gian qua nhé!', 'shared_text': '', 'time': 1617377959, 'image': 'https://scontent-sin6-4.xx.fbcdn.net/v/t1.6435-9/165943674_3584166895043499_2739462013641120862_n.png?_nc_cat=100&ccb=1-3&_nc_sid=8024bb&_nc_ohc=H7jJqOtK6i8AX-gfEME&_nc_ht=scontent-sin6-4.xx&oh=dca1a72ff916b6e950aaf63a3066e2b0&oe=60E41244', 'likes': 91, 'comments': 2, 'shares': 0, 'post_url': 'https://m.facebook.com/story.php?story_fbid=3584167855043403&id=297930300333858', 'link': None}
1


{'fail': Fals

<ipython-input-2-8f4d4a918f1b>:317: DeprecationWarning: invalid escape sequence '\/'
  return value.encode('utf-8').decode('unicode_escape').replace('\\/', '/')


{'fail': False, 'post_id': '3802893663170820', 'text': '#8231_PCs\n"Chào ac! hồi năm nhất em còn 1 môn chưa đăng ký học, em thử đăng ký học cùng D20 nhưng bị trùng tkb, bây giờ em đăng ký học hè môn đó và đã đăng ký được nhưng vì chưa từng học môn này trước đây nên em sợ sau này khi thi xong sẽ không được công nhận kết quả, có thật vậy không ạ?"\n-------------------------------------\n#V\nlink gửi cfs:https://goo.gl/9VKhVU', 'post_text': '#8231_PCs\n"Chào ac! hồi năm nhất em còn 1 môn chưa đăng ký học, em thử đăng ký học cùng D20 nhưng bị trùng tkb, bây giờ em đăng ký học hè môn đó và đã đăng ký được nhưng vì chưa từng học môn này trước đây nên em sợ sau này khi thi xong sẽ không được công nhận kết quả, có thật vậy không ạ?"\n-------------------------------------\n#V\nlink gửi cfs:https://goo.gl/9VKhVU', 'shared_text': '', 'time': 1624094709, 'image': None, 'likes': 42, 'comments': 4, 'shares': 0, 'post_url': 'https://m.facebook.com/story.php?story_fbid=3802893663170820&id=297930300333